In [12]:
!pip install flask flask-cors pyngrok tensorflow pillow numpy


In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


ngrok token

In [14]:
token_file_path = "/content/drive/Shared drives/CSDS395_Senior_Project/ngrok_token.txt"
try:
    with open(token_file_path, "r") as file:
        ngrok_token = file.read().strip()
        print("token loaded successfully")
except FileNotFoundError:
    print("no token found")

token loaded successfully


In [16]:
import tensorflow
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from PIL import Image
import io
from pyngrok import ngrok, conf


model_path = "/content/drive/Shared drives/CSDS395_Senior_Project/skin_model.h5"
model = load_model(model_path)

# init Flask
app = Flask(__name__)
CORS(app) # remove cross-origin restriction

# preprocess 224px
def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# api endpoint
@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400


    file = request.files["file"]
    img = Image.open(io.BytesIO(file.read())).convert("RGB")
    img_array = preprocess_image(img)


    predictions = model.predict(img_array)
    print("Raw model output:", predictions)
    predicted_class_idx = int(np.argmax(predictions))
    class_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

    predicted_class_name = class_labels[predicted_class_idx]


    # Print predicted class
    print(f"Predicted class: {predicted_class_name}")

    return jsonify({
        "index": predicted_class_idx,
        "label": predicted_class_name
    })

# Start Flask server
port = 5000
conf.get_default().auth_token = ngrok_token
public_url = ngrok.connect(port, hostname="noble-mature-tapir.ngrok-free.app").public_url  # ngrok tunnel
print(f" * Running on {public_url}")

app.run(port=port)


 * Running on https://noble-mature-tapir.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [05/Mar/2025 04:59:07] "POST /predict HTTP/1.1" 200 -


Raw model output: [[0.0824091  0.18020885 0.01570729 0.00223893 0.00356523 0.6651644
  0.05070617]]
Predicted class: nv
